# Sentinel-API Query

This is a program that retreives Sentinel-2 imagery over a defined (geojson) region. Users with shapefiles can easily convert regions to a json or geojson format. It is highly advised to generalize features and the API cannot handle shapes and regions with complex geometries such as fine-scale coastal boundaries. 

*Note: The API only accepts regions that are defined based on WGS 84 (EPSG:4326). Users are advised to create regions in region-specific projections and converting layers to WGS 84 in **QGIS** once finalized*.

In [ ]:
import os,sys
import numpy as np

In [ ]:
import tenacity
import logging
from sentinelsat import SentinelAPI, read_geojson, geojson_to_wkt
from datetime import date

logging.basicConfig(format='%(message)s', level='INFO')

# Please enter the user-appropriate credentials
api = SentinelAPI('USERNAME', 'PASSWORD', 'https://scihub.copernicus.eu/dhus')

# File handling, this is created to ensure file downloads continue in the case of miscommunication with servers 
@tenacity.retry(stop=tenacity.stop_after_attempt(24), wait=
                (900))
def download_all(*args, **kwargs):
    return api.download_all(*args, **kwargs)
# search by polygon, time, and SciHub query keywords
# Please make sure to specify the full path of the user-defined region
geojson_path = 'ENTER YOUR FULL GEOJSON PATH'
footprint = geojson_to_wkt(read_geojson(geojson_path))
products = api.query(footprint,
                     date=('20200901', date(2020, 9, 4)), # Note differences in date format 
                     platformname='Sentinel-2',
                     producttype='S2MSI1C',
                     cloudcoverpercentage=(0, 10)) # Cloud cover paramters can be changed based on user preference

products_df = api.to_dataframe(products)
downloaded, triggered, failed = download_all(products)
